# Introduccion a PLN Redes Neuronales Recurrentes
En la creación de redes neuronales necesitamos dos tipos de IA, para reconocer patrones o generar nuevos:

1. Las que no tienen memoria, identifica el patrón y ya!...ejemplo las de visión artificial
2. Las de memoria corta (Long Short Term Memory)...PLN
3. Las que requieren mucha memoria (aprenden casi todo...BERT)...PLN y visión artificial.


## Caso de estudio: generación de texto

Cualquier dato que se necesite procesar (sonido, imágenes, texto) primero debe ser convertido en un tensor numérico, un paso llamado “vectorización” (One-hot Encoding y WordEmbedding) de datos (y en nuestro ejemplo previamente las letras deben ser pasadas a valores numéricos

Para este ejemplo usaremos “Character level language model” propuesto por Andrej Karpathy en su artículo "The Unreasonable Effectiveness of Recurrent Neural Networks"(y parcialmente basado en su implementado en el tutorial "Generate text with an RNN" de la web de TensorFlow:

Consiste en darle a la RNN una palabra y se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores:

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles [“a”,”h”,”l”,”o”], y queremos entrenar a una RNN en la secuencia de entrenamiento “hola”. Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado: La probabilidad de “o” debería ser verosímil dada el contexto de “h”, “l” debería ser verosímil en el contexto de “ho”, y finalmente “a” debería ser también verosímil dado el contexto de “hol”.

Tambien se recomienda ver mas informacion frente al tema:

1. https://unipython.com/generacion-de-textos-con-inteligencia-artificial/
2. https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa
3. (https://www.youtube.com/watch?v=kaQCdv46OBA&ab_channel=JeffHeaton)


# Prediciendo un texto usando la herramienta de Keras.

## Instalar matplotlib en Anaconda3

Se debe tener encuenta en que perfil se va a instalar esta libreria de anaconda 3. Se recomienda para este caso de Uso de Tensor Flow, primero instalar y activar el mismo para instalar el resto de librerias con las que se va a trabajar:

1. conda install -c conda-forge matplotlib
2. conda install matplotlib
3. conda install matplotlib.pyplot
4. pip install matplotlib

## Importar Librerias

In [1]:
# manipulacion de archivos y carpetas
import io
import os
import requests # libreria que nos permite conectarnos a una direccion online o una URL

In [2]:
import matplotlib.pyplot as plt # Para trabajar los asuntos de generacion de graficos (una de varias librerias que hay)
plt.rcParams['figure.figsize'] = (16,9) #tamaño de la figura (ancho,alto)
plt.style.use('ggplot') # estilo grafico de las imagenes generadas
# guardar las imagenes y tablas en el notebook
%matplotlib inline 

In [3]:
import numpy as np
import time #Para uso del tiempo
import sys

## Instalacion de Tensor en Anaconda3

https://docs.anaconda.com/anaconda/user-guide/tasks/tensorflow/

1. Download and install Anaconda or the smaller Miniconda.

2. On Windows open the Start menu and open an Anaconda Command Prompt. On macOS or Linux open a terminal window. Use the default bash shell on macOS or Linux.

3. Choose a name for your TensorFlow environment, such as “tf”.

4. To install the current release of CPU-only TensorFlow, recommended for beginners:

In [4]:
# conda create -n tf tensorflow
# conda activate tf

Or, to install the current release of GPU TensorFlow on Linux or Windows:

In [5]:
#conda create -n tf-gpu tensorflow-gpu
#EX

### Libreria Tensor Flow

In [6]:
!pip list

Package                      Version
---------------------------- ---------
absl-py                      0.13.0
aiohttp                      3.8.1
aiosignal                    1.2.0
anyio                        2.2.0
argon2-cffi                  20.1.0
astor                        0.8.1
astunparse                   1.6.3
async-generator              1.10
async-timeout                4.0.1
attrs                        21.2.0
Babel                        2.9.1
backcall                     0.2.0
bleach                       4.0.0
blinker                      1.4
brotlipy                     0.7.0
cachetools                   4.2.2
certifi                      2021.10.8
cffi                         1.15.0
charset-normalizer           2.0.4
click                        8.0.3
colorama                     0.4.4
cryptography                 3.4.8
cycler                       0.11.0
debugpy                      1.5.1
decorator                    5.1.0
defusedxml                   0.7.1
entrypoi

In [7]:
!pip install tensorflow-gpu

In [8]:
#Librerias enfocadas al Deep Learning
import tensorflow as tf

In [9]:
print("Version: ", tf.__version__)
print("GPU esta", "disponible" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print("Dispositivos disponibles: ", tf.config.list_physical_devices())

Version:  2.7.0
GPU esta disponible
Dispositivos disponibles:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Uso de la GPU en Tensor Flow

Uso de GPU para entrenar en tensorflow

https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa

Elejir una de las siguientes dos:

In [11]:
tf.device('/GPU:0') #activando la GPU

## Directorio en PC

In [12]:
# directorio donde guardar los checkpoints en pc
root = 'C:/Users/Felip/Desktop/checkpoints'

## Descargar y preprocesar los datos

In [13]:
fileDL= tf.keras.utils.get_file('[100 LETRAS] Regueton bien sucio.txt','https://raw.githubusercontent.com/iFePax/DeepLearning/main/Datasets/%5B100%20LETRAS%5D%20Regueton%20bien%20sucio.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')

In [14]:
# Si se quiere mostrar el texto que ha sido cargado
print(texto)

[Darell]
Este es el verdadero remix, baby

[Ozuna]
Paso muchas noches pensándote
Yo no sé ni cómo, ni cuándo fue
Pero solo sé que yo recordé
Cómo te lo hacía yo aquella vez
Y yo no puedo seguir solo, pero sé
Que te boté

De mi vida te boté, y te boté
Te di banda y te solté, yo te solté
Pa'l carajo usted se fue, y usted se fue
De mi vida te boté, yo te boté, yeh-yeh, mami

[Bad Bunny]
Baby, la vida es un ciclo
Y lo que no sirve ya no lo reciclo
Así que de mi vida muévete
Que si te lo meto, es pa' recordar un TBT, yeh

Ya yo me cansé de tus mentira'
Ahora hay una más dura que me tira
Todo tiene su final, todo expira
Tú eres pasado y el pasado nunca vira

Arranca pa'l carajo, mi cuerpo no te necesita
Lo que pide es un perreo sucio en La Placita
No creo que lo nuestro se repita
Dale, prende un Phillie, deja uno ready pa' ahorita, yeh

Odio saber que en ti una vez más yo confié
Odio todos los te amo que mil veces te texteé
Baby, mejor que tú, ahora tengo como die'
Lo nuestro iba en Bugatti 

## Limpiar y normalizar el texto

In [15]:
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto esta compuesto de estos :99 caracteres
['\n', '\r', ' ', '!', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '¿', 'Á', 'É', 'Ñ', 'Ú', 'á', 'é', 'ë', 'í', 'ñ', 'ó', 'ú', 'ü', 'ō', 'е', '\u2005', '‘', '’', '\u205f']


In [16]:
import re
from unicodedata import normalize
#pasa todo a minuscula
texto     = texto.lower()
#reemplazar tildes por letras similares sin tildes
transfor  = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
texto     = normalize('NFKC', normalize('NFKD', texto).translate(transfor))
#quitar saltos de linea
texto      = texto.strip()
texto      = re.sub('\r|\n', ' ',texto)
#quitar espacios dobles
texto      = re.sub(' +', ' ', texto)
#quitando caracteres especiales
texto = re.sub(r"[^a-zA-Z0-9]+"," ",texto)
print(texto)

 darell este es el verdadero remix baby ozuna paso muchas noches pensandote yo no se ni como ni cuando fue pero solo se que yo recorde como te lo hacia yo aquella vez y yo no puedo seguir solo pero se que te bote de mi vida te bote y te bote te di banda y te solte yo te solte pa l carajo usted se fue y usted se fue de mi vida te bote yo te bote yeh yeh mami bad bunny baby la vida es un ciclo y lo que no sirve ya no lo reciclo asi que de mi vida muevete que si te lo meto es pa recordar un tbt yeh ya yo me canse de tus mentira ahora hay una mas dura que me tira todo tiene su final todo expira tu eres pasado y el pasado nunca vira arranca pa l carajo mi cuerpo no te necesita lo que pide es un perreo sucio en la placita no creo que lo nuestro se repita dale prende un phillie deja uno ready pa ahorita yeh odio saber que en ti una vez mas yo confie odio todos los te amo que mil veces te textee baby mejor que tu ahora tengo como die lo nuestro iba en bugatti y te quedaste a pie yo te bote te 

## Entendiendo El Texto

In [17]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:220975 caracteres
el texto esta compuesto de estos :37 caracteres
[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [18]:
print(texto)

 darell este es el verdadero remix baby ozuna paso muchas noches pensandote yo no se ni como ni cuando fue pero solo se que yo recorde como te lo hacia yo aquella vez y yo no puedo seguir solo pero se que te bote de mi vida te bote y te bote te di banda y te solte yo te solte pa l carajo usted se fue y usted se fue de mi vida te bote yo te bote yeh yeh mami bad bunny baby la vida es un ciclo y lo que no sirve ya no lo reciclo asi que de mi vida muevete que si te lo meto es pa recordar un tbt yeh ya yo me canse de tus mentira ahora hay una mas dura que me tira todo tiene su final todo expira tu eres pasado y el pasado nunca vira arranca pa l carajo mi cuerpo no te necesita lo que pide es un perreo sucio en la placita no creo que lo nuestro se repita dale prende un phillie deja uno ready pa ahorita yeh odio saber que en ti una vez mas yo confie odio todos los te amo que mil veces te textee baby mejor que tu ahora tengo como die lo nuestro iba en bugatti y te quedaste a pie yo te bote te 

## Pasar el texto a numeros

Las redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [19]:
# ordena los caracteres presentes en el texto de forma alfabetica como en una lista
# a cada uno de los caracteres listados asigname un numero ordenado

In [20]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: ' darell este es el verdadero remix baby ozuna paso muchas noches pensandote yo no se ni como ni cuan'
array([ 0, 14, 11, 28, 15, 22, 22,  0, 15, 29, 30, 15,  0, 15, 29,  0, 15,
       22,  0, 32, 15, 28, 14, 11, 14, 15, 28, 25,  0, 28, 15, 23, 19, 34,
        0, 12, 11, 12, 35,  0, 25, 36, 31, 24, 11,  0, 26, 11, 29, 25,  0,
       23, 31, 13, 18, 11, 29,  0, 24, 25, 13, 18, 15, 29,  0, 26, 15, 24,
       29, 11, 24, 14, 25, 30, 15,  0, 35, 25,  0, 24, 25,  0, 29, 15,  0,
       24, 19,  0, 13, 25, 23, 25,  0, 24, 19,  0, 13, 31, 11, 24])


In [21]:
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

## Preparar los datos para ser usados en la RNN

In [22]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=350
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

' darell este es el verdadero remix baby ozuna paso muchas noches pensandote yo no se ni como ni cuando fue pero solo se que yo recorde como te lo hacia yo aquella vez y yo no puedo seguir solo pero se que te bote de mi vida te bote y te bote te di banda y te solte yo te solte pa l carajo usted se fue y usted se fue de mi vida te bote yo te bote yeh '
'yeh mami bad bunny baby la vida es un ciclo y lo que no sirve ya no lo reciclo asi que de mi vida muevete que si te lo meto es pa recordar un tbt yeh ya yo me canse de tus mentira ahora hay una mas dura que me tira todo tiene su final todo expira tu eres pasado y el pasado nunca vira arranca pa l carajo mi cuerpo no te necesita lo que pide es un per'
'reo sucio en la placita no creo que lo nuestro se repita dale prende un phillie deja uno ready pa ahorita yeh odio saber que en ti una vez mas yo confie odio todos los te amo que mil veces te textee baby mejor que tu ahora tengo como die lo nuestro iba en bugatti y te quedaste a pie yo te b

### Separar los datos en agrupamientos (batches)

In [23]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)

for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  ' darell este es el verdadero remix baby ozuna paso muchas noches pensandote yo no se ni como ni cuando fue pero solo se que yo recorde como te lo hacia yo aquella vez y yo no puedo seguir solo pero se que te bote de mi vida te bote y te bote te di banda y te solte yo te solte pa l carajo usted se fue y usted se fue de mi vida te bote yo te bote yeh'
Target data:  'darell este es el verdadero remix baby ozuna paso muchas noches pensandote yo no se ni como ni cuando fue pero solo se que yo recorde como te lo hacia yo aquella vez y yo no puedo seguir solo pero se que te bote de mi vida te bote y te bote te di banda y te solte yo te solte pa l carajo usted se fue y usted se fue de mi vida te bote yo te bote yeh '


In [24]:
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )

In [25]:
#imprimimos el tensor del dataset
print(dataset)

<MapDataset shapes: ((350,), (350,)), types: (tf.int32, tf.int32)>


In [26]:
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 128
#    -Tamaño de memoria disponible 
BUFFER_SIZE=100000

In [27]:
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)

<BatchDataset shapes: ((128, 350), (128, 350)), types: (tf.int32, tf.int32)>


In [28]:
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

# Construcción del modelo RNN

Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.

In [29]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model

vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim=1024
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 1024)         37888     
                                                                 
 lstm (LSTM)                 (128, None, 1024)         8392704   
                                                                 
 dense (Dense)               (128, None, 37)           37925     
                                                                 
Total params: 8,468,517
Trainable params: 8,468,517
Non-trainable params: 0
_________________________________________________________________


## Creando chekpoints

Una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.

In [30]:
print(root)

C:/Users/Felip/Desktop/checkpoints


In [31]:
checkpoint_dir=root

Se utiliza un checkpoint_dir en un espacio en DRIVE para mejorar la situacion del entranimiento. Google Colab ofrece un espacio en disco limitado para guardar variables, cada checkpoint en este proceso en concreto puede llegar a pesar 60 a 70MB, con un entrenamiento relativamente basico de 1000 Epochs, tendriamos un espacio de 70000MB o 70Gb en disco de los cuales requerir.

In [32]:
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')

# Entrenando la RNN

## Entrenamiento utilizando Checkpoint

Se recomienda investigar los siguientes enlaces de informacion:

1. https://keras.io/api/callbacks/model_checkpoint/
2. https://towardsdatascience.com/checkpointing-deep-learning-models-in-keras-a652570b8de6

In [33]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:52:33_Pacific_Standard_Time_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


In [34]:
# ciclos de entrenamiento = 500
# verbose 0 = no quiero ver por terminal la descripcion de cada ciclo de entrenamiento
# vervose 0 = menor requerimiento de recursos en el entrenamiento, se utiliza netamiente la GPU para entrenar

EPOCHS=2500
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/2500
4/4 [==============================] - ETA: 0s - loss: 3.5571 - accuracy: 0.1424
Epoch 00001: loss improved from inf to 3.55710, saving model to C:/Users/Felip/Desktop/checkpoints\cp_0001.ckpt
4/4 [==============================] - 6s 682ms/step - loss: 3.5571 - accuracy: 0.1424
Epoch 2/2500
4/4 [==============================] - ETA: 0s - loss: 3.0468 - accuracy: 0.1420
Epoch 00002: loss improved from 3.55710 to 3.04683, saving model to C:/Users/Felip/Desktop/checkpoints\cp_0002.ckpt
4/4 [==============================] - 3s 631ms/step - loss: 3.0468 - accuracy: 0.1420
Epoch 3/2500
4/4 [==============================] - ETA: 0s - loss: 2.7544 - accuracy: 0.2332
Epoch 00003: loss improved from 3.04683 to 2.75443, saving model to C:/Users/Felip/Desktop/checkpoints\cp_0003.ckpt
4/4 [==============================] - 3s 664ms/step - loss: 2.7544 - accuracy: 0.2332
Epoch 4/2500
4/4 [==============================] - ETA: 0s - loss: 2.6767 - accuracy: 0.2161
Epoch 00004: loss i

## Comenzar entrenamiento desde el ultimo checkpoint
Desde la carpeta que optamos guardar los checkpoints

*   el archivo .data es el archivo que contiene nuestras variables de entrenamiento y vamos a ir tras él.
*   el archivo checkpoint, simplemente mantiene un registro de los últimos archivos de punto de control guardados

In [41]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model2=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

C:/Users/Felip/Desktop/checkpoints\cp_0238.ckpt


In [42]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model2.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history_V2=model2.fit(dataset, 
                    epochs=256, 
                    verbose=1,
                    callbacks=[cp_callback])

Epoch 1/256
4/4 [==============================] - ETA: 0s - loss: 0.0194 - accuracy: 0.9942
Epoch 00001: loss improved from 0.01982 to 0.01944, saving model to C:/Users/Felip/Desktop/checkpoints\cp_0001.ckpt
4/4 [==============================] - 4s 534ms/step - loss: 0.0194 - accuracy: 0.9942
Epoch 2/256
4/4 [==============================] - ETA: 0s - loss: 0.0215 - accuracy: 0.9939
Epoch 00002: loss did not improve from 0.01944
4/4 [==============================] - 2s 461ms/step - loss: 0.0215 - accuracy: 0.9939
Epoch 3/256
4/4 [==============================] - ETA: 0s - loss: 0.0206 - accuracy: 0.9942
Epoch 00003: loss did not improve from 0.01944
4/4 [==============================] - 2s 452ms/step - loss: 0.0206 - accuracy: 0.9942
Epoch 4/256
4/4 [==============================] - ETA: 0s - loss: 0.0204 - accuracy: 0.9943
Epoch 00004: loss did not improve from 0.01944
4/4 [==============================] - 2s 463ms/step - loss: 0.0204 - accuracy: 0.9943
Epoch 5/256
4/4 [======

In [43]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

C:/Users/Felip/Desktop/checkpoints\cp_0001.ckpt


## Generando Texto usando RNN

In [44]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [45]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 1 #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

### Generando texto

In [46]:
print(generate_text(model, start_string="perreo"))

perreo sucio en la placita no creo que lo nuestro se repita dale prende un phillie deja uno ready pa ahorita yeh odio saber que en ti una vez mas yo confie odio todos los te amo que mil veces te textee baby mejor que tu ahora tengo como die lo nuestro iba en bugatti y te quedaste a pie yo te bote te di banda y te solte yo te solte pa l carajo te mande yo te mande y de mi vida se note me olvido de to menos de ese culote los recuerdos mas cabron los teniamos en el bote te quedaste al lado mio y no volvi


# Exportando modelo

Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).

In [47]:
from keras.models import model_from_json
import os

In [48]:
dir_export= 'C:/Users/Felip/Documents/Deep Learning/exports/'

In [49]:
print(dir_export)

C:/Users/Felip/Documents/Deep Learning/exports/


In [50]:
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_Regueton-sucio-v1.json'), 'w') as json_file:
    json_file.write(model_json)

    
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_Regueton-sucio-v1.hdf5'))
model.save(os.path.join(dir_export,'RNN_Regueton-sucio-v1model.h5'))
print("modelo salvado en Directorio Local")

modelo salvado en Directorio Local


# Cargando modelo serializado

## Descargar el modelo usando PYRIND & URLLIB

In [ ]:
!pip install pyprind

  Using cached PyPrind-2.11.3-py2.py3-none-any.whl (8.4 kB)


In [ ]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


In [ ]:
import urllib.request
url_github_Model='https://github.com/iFePax/DeepLearning/final-project/export'
urllib.request.urlretrieve(url_github_Model,
                           'RNN_Regueton-1_model.h5', 
                           reporthook)

HTTPError: HTTP Error 404: Not Found

In [ ]:
new_model = tf.keras.models.load_model('C:/Users/ruiso/Desktop/EscritorioAnaconda3Python/Deep 2/RNN_ElResplandor_model.h5')

OSError: SavedModel file does not exist at: C:/Users/ruiso/Desktop/EscritorioAnaconda3Python/Deep 2/RNN_ElResplandor_model.h5\{saved_model.pbtxt|saved_model.pb}

In [ ]:
print(generate_text(model, start_string=u"tienes que matarlo"))